In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-30/validation_data.csv
/kaggle/input/data-storm-30/train_data.csv
/kaggle/input/data-storm-30/test_data.csv
/kaggle/input/data-storm-30/DataStorm3.0_KaggleProblemStatement.pdf


# Sales Demand Forecasting

### Business Problem
Forecasting sales is one of the most fundamental problems most retail chains have

### Use-case Definition
Retail Chain A has used traditional forecasting methods to estimate projected sales for each
item across stores, But this method in not accurate.Therefore they need to improve forecasting through advanced analytics and machine learning techniques.

### Data Description
CategoryCode: The items are classified into 4 different categories; this column refers to its code.

ItemCode: An identifier for each item

DateID: The date for the respective sales value

DailySales: Sales value in quantity units for the respective item on the given date.

Week:

WeeklySales: Sales value in quantity units for the respective item for the given week.


# Import the data

In [2]:
tr_sales=pd.read_csv('../input/data-storm-30/train_data.csv')
val_sales=pd.read_csv('../input/data-storm-30/validation_data.csv')

### Validation Data

In [3]:
val_sales.head()# Check the first five data

,CategoryCode,ItemCode,Week,WeeklySales
0,category_2,1044502,w1,11
1,category_2,1105009,w1,11
2,category_2,913561,w4,5
3,category_1,1048975,w4,30
4,category_1,17287,w2,60


In [4]:
# Check the first five data
val_sales['Week'].unique(),val_sales.shape

(array(['w1', 'w4', 'w2', 'w3'], dtype=object), (370, 4))

In [5]:
#Convert 
val_sales['Week'].replace(to_replace = ['w1', 'w2','w3','w4'],value =['2/20/2022','2/27/2022','3/6/2022','3/13/2022'],inplace=True)
val_sales['Date'] = pd.to_datetime(val_sales['Week'])
val_sales = val_sales[['CategoryCode','ItemCode','Date','WeeklySales']]
val_sales.head()

,CategoryCode,ItemCode,Date,WeeklySales
0,category_2,1044502,2022-02-20,11
1,category_2,1105009,2022-02-20,11
2,category_2,913561,2022-03-13,5
3,category_1,1048975,2022-03-13,30
4,category_1,17287,2022-02-27,60


In [6]:
val_sales['month']=val_sales['Date'].dt.month
val_sales['day']=val_sales['Date'].dt.day
val_sales.head()

,CategoryCode,ItemCode,Date,WeeklySales,month,day
0,category_2,1044502,2022-02-20,11,2,20
1,category_2,1105009,2022-02-20,11,2,20
2,category_2,913561,2022-03-13,5,3,13
3,category_1,1048975,2022-03-13,30,3,13
4,category_1,17287,2022-02-27,60,2,27


In [7]:
#Dummies = pd.get_dummies(val_sales.CategoryCode)
#val_sales_Merged=pd.concat([val_sales.drop('CategoryCode',axis='columns'),Dummies.drop('category_4',axis='columns')],axis='columns')
#val_sales_Merged

In [8]:
# Validation Split
val_Xtes = val_sales[['ItemCode','month','day']].values#.reshape(-1, 1) #'category_1','category_2','category_3'
val_ytes = val_sales['WeeklySales'].values#.reshape(-1, 1)

### Train Data

In [9]:
tr_sales.head()

,CategoryCode,ItemCode,DateID,DailySales
0,category_2,117610,11/6/2021,7
1,category_4,836584,11/18/2021,16
2,category_1,370195,1/24/2022,6
3,category_2,172582,10/30/2021,5
4,category_2,1006009,10/30/2021,5


In [10]:
tr_sales.isnull().sum()

CategoryCode    0
ItemCode        0
DateID          0
DailySales      0
dtype: int64

In [11]:
tr_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19921 entries, 0 to 19920
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CategoryCode  19921 non-null  object
 1   ItemCode      19921 non-null  int64 
 2   DateID        19921 non-null  object
 3   DailySales    19921 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 622.7+ KB


In [12]:
#Change DateID to date time
tr_sales['Date'] = pd.to_datetime(tr_sales['DateID'])
#tr_sales['Year'] = pd.to_datetime(tr_sales['Date']).dt.year
#tr_sales['Month'] = pd.to_datetime(tr_sales['Date']).dt.month
#tr_sales['Week'] = pd.to_datetime(tr_sales['Date']).dt.week
# tr_sales['Day'] = pd.to_datetime(tr_sales['Date']).dt.day
# tr_sales.sort_values(by=['Date'], inplace=True, ascending=True)
trr_sales=tr_sales[['CategoryCode','ItemCode','Date','DailySales']]
#trr_sales['CategoryCode'].replace(to_replace = ['category_1', 'category_2','category_3','category_4'],value =[1,2,3,4],inplace=True)
trr_sales.head()

,CategoryCode,ItemCode,Date,DailySales
0,category_2,117610,2021-11-06,7
1,category_4,836584,2021-11-18,16
2,category_1,370195,2022-01-24,6
3,category_2,172582,2021-10-30,5
4,category_2,1006009,2021-10-30,5


In [13]:
t = trr_sales[['CategoryCode','ItemCode']]

In [14]:
#Convert CategoryCode to number
#trrCat_sales = trr_sales['CategoryCode'].replace(to_replace = ['category_1', 'category_2','category_3','category_4'],value =[1,2,3,4],inplace=True)
trr_sales = trr_sales[['ItemCode','Date','DailySales']]
trr_sales.head()

,ItemCode,Date,DailySales
0,117610,2021-11-06,7
1,836584,2021-11-18,16
2,370195,2022-01-24,6
3,172582,2021-10-30,5
4,1006009,2021-10-30,5


In [15]:
tr_weekly_data = trr_sales.groupby("ItemCode",as_index=False).resample('W-MON',on='Date').sum().reset_index()#.sort_values(by='Date')
#tr_weekly_data = weekly_data[['ItemCode','Date','DailySales']]
tr_weekly_data.head()

,level_0,Date,ItemCode,DailySales
0,0,2021-10-04,13672,31
1,0,2021-10-11,20508,42
2,0,2021-10-18,20508,53
3,0,2021-10-25,23926,40
4,0,2021-11-01,23926,39


In [16]:
tr_weekly_data['WeeklySales'] = tr_weekly_data['DailySales']
train_weekly_data=tr_weekly_data[['Date','ItemCode','WeeklySales']]
train_weekly_data.head()

,Date,ItemCode,WeeklySales
0,2021-10-04,13672,31
1,2021-10-11,20508,42
2,2021-10-18,20508,53
3,2021-10-25,23926,40
4,2021-11-01,23926,39


In [17]:
train_weekly_data['month']=train_weekly_data['Date'].dt.month
train_weekly_data['day']=train_weekly_data['Date'].dt.day
#train_weekly_data

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
#Train data split
tra_Xtr = train_weekly_data[['ItemCode','month','day']].values#.reshape(-1, 1) #'category_1','category_2','category_3'
tra_ytr = train_weekly_data['WeeklySales'].values#.reshape(-1, 1)
tra_Xtr.shape ,tra_ytr.shape

((3837, 3), (3837,))

In [19]:
train_weekly_data['WeeklySales'].describe()#Summary Statistics for Weekly sales data

count    3837.000000
mean       38.169664
std        73.933827
min         0.000000
25%         9.000000
50%        18.000000
75%        38.000000
max       862.000000
Name: WeeklySales, dtype: float64

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,Y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores=[]
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algorithm_name, config in algos.items():
        grid_search =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        grid_search.fit(X,Y)
        scores.append({
            'model': algorithm_name,
            'best_score': grid_search.best_score_,
            'best_params': grid_search.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(tra_Xtr, tra_ytr)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pi

,model,best_score,best_params
0,linear_regression,-0.001684,{'normalize': False}
1,lasso,-0.001452,"{'alpha': 2, 'selection': 'cyclic'}"
2,decision_tree,0.613336,"{'criterion': 'friedman_mse', 'splitter': 'best'}"


In [22]:
from sklearn.tree import DecisionTreeRegressor
Dt = DecisionTreeRegressor()
Dt.fit(tra_Xtr, tra_ytr)
DT_pred = Dt.predict(val_Xtes)
#print(lm_pred)
Dt.score(tra_Xtr, tra_ytr)

1.0

In [23]:
#Total Mean Absolute Percentage Error (Total MAPE) for evaluation.
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(val_ytes, DT_pred)

1.1417827291525682

In [24]:
test_sales=pd.read_csv('../input/data-storm-30/test_data.csv')
test_sales.head()

,CategoryCode,ItemCode,Week,PredictedSales
0,category_1,43738,w4,NaN
1,category_2,1006090,w1,NaN
2,category_2,1076929,w4,NaN
3,category_1,1081321,w3,NaN
4,category_2,216151,w4,NaN


In [25]:
#Convert 
test_sales['Week'].replace(to_replace = ['w1', 'w2','w3','w4'],value =['2/20/2022','2/27/2022','3/6/2022','3/13/2022'],inplace=True)
test_sales['Date'] = pd.to_datetime(test_sales['Week'])
test_sales['month']=test_sales['Date'].dt.month
test_sales['day']=test_sales['Date'].dt.day
test_sales = test_sales[['ItemCode','month','day']]
test_sales.head()

,ItemCode,month,day
0,43738,3,13
1,1006090,2,20
2,1076929,3,13
3,1081321,3,6
4,216151,3,13


In [26]:
test=pd.read_csv('../input/data-storm-30/test_data.csv')
test = test[['CategoryCode','ItemCode','Week']]

In [27]:
Dt_pred = Dt.predict(test_sales)
Dt_pr = pd.DataFrame(Dt_pred,columns =['PredictedSales'])

PredictedSales = pd.concat([test,Dt_pr],axis=1)
PredictedSales

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:439: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


,CategoryCode,ItemCode,Week,PredictedSales
0,category_1,43738,w4,22.0
1,category_2,1006090,w1,1.0
2,category_2,1076929,w4,12.0
3,category_1,1081321,w3,12.0
4,category_2,216151,w4,72.0
...,...,...,...,...
372,category_2,1101571,w1,1.0
373,category_2,1090258,w4,2.0
374,category_2,906595,w1,2.0
375,category_2,32245,w1,38.0


In [28]:
cols = ['CategoryCode', 'ItemCode', 'Week']
PredictedSales['ID'] = PredictedSales[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
PredictedSalesforSub=PredictedSales[['ID','PredictedSales']]
PredictedSalesforSub

,ID,PredictedSales
0,category_1_43738_w4,22.0
1,category_2_1006090_w1,1.0
2,category_2_1076929_w4,12.0
3,category_1_1081321_w3,12.0
4,category_2_216151_w4,72.0
...,...,...
372,category_2_1101571_w1,1.0
373,category_2_1090258_w4,2.0
374,category_2_906595_w1,2.0
375,category_2_32245_w1,38.0


In [29]:
PredictedSalesforSub.to_csv('PredictedSales.csv')